In [10]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sys
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import numpy as np
from numpy import zeros,ones,linalg
from sklearn.cross_validation import train_test_split
import mosek
import math
import sys

from skimage import measure
from skimage.draw import ellipsoid

c = [10,10,10]
r=10

ellip_base = np.zeros((100,100,100))
for i in range(ellip_base.shape[0]):
    for j in range(ellip_base.shape[1]):
        for k in range(ellip_base.shape[2]):
            if (i-c[0])**2 + (j-c[1])**2 + (k-c[2])**2 > r**2:
                ellip_base[i][j][k] = .1
            if (i-c[0])**2 + (j-c[1])**2 + (k-c[2])**2 < r**2:
                ellip_base[i][j][k] = -.1
            if (i-c[0])**2 + (j-c[1])**2 + (k-c[2])**2 == r**2:
                ellip_base[i][j][k] = 0


# Define a stream printer to grab output from MOSEK
def streamprinter(text):
#     sys.stdout.write(text)
#     sys.stdout.flush()
    pass


def kernel(x1,x2):
    return math.exp(-1*math.pow(linalg.norm(x1-x2),2)/(2*math.pow(sigma,
                                                                  2)))

sigma = 100

x = []
f = open('bunny.obj.txt')
for line in f:
    line = line.strip()
    if line != '' and line[0] != '#':
        line_split = line.split()
        if len(line_split)==4 and line_split[0] == 'v':
            x.append(line_split[1:])
x = np.array(x)
x=x.astype(np.float)
y = np.ones(len(x))
m = len(x)
# print 'points:',m
x,ignore_x,y,ignore_y=train_test_split(x,y,train_size=0.02,random_state=8)
m = len(x)

x0_max = np.amax(x[:,0])+0.1
x0_min = np.amin(x[:,0])-0.1
x1_max = np.amax(x[:,1])+0.1
x1_min = np.amin(x[:,1])-0.1
x2_max = np.amax(x[:,2])+0.1
x2_min = np.amin(x[:,2])-0.1


print x0_max 
print x0_min 
print x1_max 
print x1_min 
print x2_max 
print x2_min 

# print 'points:',m

# overfit/underfit param
v=.99

# Open MOSEK and create an environment and task
# Make a MOSEK environment
env = mosek.Env ()
# Attach a printer to the environment
env.set_Stream (mosek.streamtype.log, streamprinter)
# Create a task
task = env.Task()
task.set_Stream (mosek.streamtype.log, streamprinter)
# Set up and input bounds and linear coefficients
bkc   = [ mosek.boundkey.fx]
blc   = [ 1.0 ]
buc   = [ 1.0 ]

bkx=[]
for i in range(m):
    bkx.append(mosek.boundkey.ra)
bux=(ones((m))) * (1/(v*m))
blx=(ones((m))) * (-1/(v*m))
c=(zeros((m)))

numvar = len(bkx)
numcon = len(bkc)

# Append 'numcon' empty constraints.
# The constraints will initially have no bounds.
task.appendcons(numcon)

# Append 'numvar' variables.
# The variables will initially be fixed at zero (x=0).
task.appendvars(numvar)

for j in range(numvar):
    # Set the linear term c_j in the objective.
    task.putcj(j,c[j])
    # Set the bounds on variable j
    # blx[j] <= x_j <= bux[j]
    task.putbound(mosek.accmode.var,j,bkx[j],blx[j],bux[j])

    # Input i, j of A
    task.putaij(0, j, 1)

for i in range(numcon):
    task.putbound(mosek.accmode.con,i,bkc[i],blc[i],buc[i])

# Input the objective sense (minimize/maximize)
task.putobjsense(mosek.objsense.maximize)

# Set up and input quadratic objective
for i in range(m):
    for j in range(i+1):
        task.putqobjij(i,j,kernel(x[i],x[j]))

task.putobjsense(mosek.objsense.minimize)

# Optimize
task.optimize()
# Print a summary containing information
# about the solution for debugging purposes
task.solutionsummary(mosek.streamtype.msg)

prosta = task.getprosta(mosek.soltype.itr)
solsta = task.getsolsta(mosek.soltype.itr)

# Output a solution
xx = zeros(numvar, float)
task.getxx(mosek.soltype.itr, xx)

# if solsta == mosek.solsta.optimal or solsta == mosek.solsta.near_optimal:
#     print("Optimal solution: %s" % xx)
# elif solsta == mosek.solsta.dual_infeas_cer:
#     print("Primal or dual infeasibility.\n")
# elif solsta == mosek.solsta.prim_infeas_cer:
#     print("Primal or dual infeasibility.\n")
# elif solsta == mosek.solsta.near_dual_infeas_cer:
#     print("Primal or dual infeasibility.\n")
# elif  solsta == mosek.solsta.near_prim_infeas_cer:
#     print("Primal or dual infeasibility.\n")
# elif mosek.solsta.unknown:
#     print("Unknown solution status")
# else:
#     print("Other solution status")

alpha = xx

x_test = x[0] #test at x
w_x=0
for i in range(m):
    w_x = w_x + (alpha[i]*kernel(x[i],x_test))

# print 'w = ',w_x

num_steps = 50
x0_range = np.linspace(x0_min, x0_max,num_steps)
x1_range = np.linspace(x1_min, x1_max,num_steps)
x2_range = np.linspace(x2_min, x2_max,num_steps)

# xx,yy,zz = np.meshgrid(x0_range,x1_range,x2_range)

# print xx,yy,zz

data = np.zeros((num_steps,num_steps,num_steps))

for i in range(num_steps):
    for j in range(num_steps):
        for k in range(num_steps):
            x_temp = np.asarray([x0_range[i],
                                x1_range[j],
                                x2_range[k]])
            w_x=0
            for i2 in range(m):
                w_x = w_x + (alpha[i2]*kernel(x[i2],x_temp))
            data[i,j,k] = w_x
            

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sys
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

from skimage import measure
from skimage.draw import ellipsoid

# Generate a level set about zero of two identical ellipsoids in 3D
# ellip_base = ellipsoid(6, 10, 16, levelset=True)

# ellip_base[np.where(ellip_base == False)] = 1
# ellip_base[np.where(ellip_base == True)] = -1

# print ellip_base
# print 
# print ellip_base.shape

# for i in range(ellip_base.shape[0]):
#     for j in range(ellip_base.shape[1]):
#         for k in range(ellip_base.shape[2]):
#       if ellip_base[i][j][k] != False and ellip_base[i][j][k] != True:
#                 print ellip_base[i][j][k]
# return

# print ellip_base.shape

# ellip_double = np.concatenate((ellip_base[:-1, ...],
#                                ellip_base[2:, ...]), axis=0)

# print ellip_base[0][0].shape
# print ellip_base[:-1, ...].shape
# print ellip_base[2:, ...].shape
# print ellip_base[0][0].shape
# print ellip_double.shape
# print ellip_double[0]

print data

# Use marching cubes to obtain the surface mesh of these ellipsoids
verts, faces = measure.marching_cubes(data, 0)

# Display resulting triangular mesh using Matplotlib.This can also be done
# with mayavi (see skimage.measure.marching_cubes docstring).
fig = plt.figure(figsize=(10, 12))
ax = fig.add_subplot(111, projection='3d')

# Fancy indexing: `verts[faces]` to generate a collection of triangles
mesh = Poly3DCollection(verts[faces])
ax.add_collection3d(mesh)

ax.set_xlabel("x-axis: a = 6 per ellipsoid")
ax.set_ylabel("y-axis: b = 10")
ax.set_zlabel("z-axis: c = 16")

ax.set_xlim(0, 24)  # a = 6 (times two for 2nd ellipsoid)
ax.set_ylim(0, 20)  # b = 10
ax.set_zlim(0, 32)  # c = 16

plt.show()


0.15988667
-0.1902417
0.27456543
-0.06633634
0.15342353
-0.13901333
[[[ 0.99999595  0.99999603  0.99999612 ...,  0.99999619  0.99999611
    0.99999602]
  [ 0.99999606  0.99999614  0.99999623 ...,  0.9999963   0.99999622
    0.99999613]
  [ 0.99999616  0.99999625  0.99999633 ...,  0.9999964   0.99999632
    0.99999624]
  ..., 
  [ 0.99999589  0.99999598  0.99999606 ...,  0.99999614  0.99999605
    0.99999597]
  [ 0.99999577  0.99999586  0.99999594 ...,  0.99999602  0.99999594
    0.99999585]
  [ 0.99999565  0.99999574  0.99999582 ...,  0.9999959   0.99999582
    0.99999573]]

 [[ 0.99999606  0.99999615  0.99999623 ...,  0.99999631  0.99999623
    0.99999614]
  [ 0.99999617  0.99999626  0.99999634 ...,  0.99999642  0.99999634
    0.99999625]
  [ 0.99999628  0.99999637  0.99999645 ...,  0.99999652  0.99999644
    0.99999636]
  ..., 
  [ 0.99999601  0.9999961   0.99999618 ...,  0.99999625  0.99999617
    0.99999609]
  [ 0.99999589  0.99999598  0.99999606 ...,  0.99999614  0.99999606
    0.

ValueError: Contour level must be within volume data range.

In [8]:

print x0_max 
print x0_min 
print x1_max 
print x1_min 
print x2_max 
print x2_min 

print data
print data.shape

print alpha


0.05988667
-0.0902417
0.17456543
0.03366366
0.05342353
-0.03901333
[[[ 0.99984776  0.99984848  0.99984916 ...,  0.99984302  0.99984206
    0.99984107]
  [ 0.99984954  0.99985026  0.99985094 ...,  0.9998448   0.99984385
    0.99984286]
  [ 0.99985124  0.99985196  0.99985264 ...,  0.9998465   0.99984555
    0.99984456]
  ..., 
  [ 0.99984215  0.99984287  0.99984355 ...,  0.99983741  0.99983645
    0.99983546]
  [ 0.99984005  0.99984077  0.99984145 ...,  0.99983531  0.99983435
    0.99983336]
  [ 0.99983786  0.99983858  0.99983926 ...,  0.99983312  0.99983217
    0.99983118]]

 [[ 0.99985001  0.99985073  0.99985141 ...,  0.99984527  0.99984432
    0.99984333]
  [ 0.9998518   0.99985251  0.99985319 ...,  0.99984706  0.9998461
    0.99984511]
  [ 0.99985349  0.99985421  0.99985489 ...,  0.99984875  0.9998478
    0.99984681]
  ..., 
  [ 0.99984441  0.99984512  0.99984581 ...,  0.99983966  0.99983871
    0.99983772]
  [ 0.9998423   0.99984302  0.9998437  ...,  0.99983756  0.99983661
    0.999